In [3]:


import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")

# Normalización
from sklearn.preprocessing import StandardScaler

# Reducción de Dimensionalidad
from umap import UMAP
from sklearn.decomposition import PCA

# Clustering
from sklearn_extra.cluster import KMedoids

# Metricas
from sklearn.metrics import rand_score, adjusted_rand_score
from sklearn.metrics import silhouette_samples, silhouette_score

/home/nachengue/fundar/python_projects/unsupervised-rice-image-segmentation/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-10 18:52:11.170422: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-10 18:52:11.173564: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-10 18:52:11.221566: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-10 18:52:11.222650: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized t

In [2]:
def calcula_sse_kmedoids(KM,d):
  se = []
  for i in range(0,KM.n_clusters):
    se.append(sum(d[KM.medoid_indices_[i],KM.labels_==i]**2))
  return sum(se)

def cross_tab(Labels_orig, Labels_clust):
    '''crea matriz de confusión para evaluar etiquetado
    labels_orig  = etiquetas originales - reales
    labels_test  = etiquetas halladeas por el algoritmo'''
    tmp = pd.DataFrame({'Labels_orig': Labels_orig, 'Labels_clust': Labels_clust})
    ct = pd.crosstab(tmp['Labels_clust'],tmp['Labels_orig']) # Create crosstab: ct
    rand = rand_score(Labels_orig, Labels_clust)
    arand= adjusted_rand_score(Labels_orig, Labels_clust)
    print(f'RAND score={rand:.4f}, Ajusted RAND={arand:.4f}, cantidad_de_muestras={len(Labels_orig):,d}')
    return ct

In [4]:
# Selecciono N filas de cada archivo input. 

DEFAULT_LENGTH = 15000 
SAMPLE_LENGTH = 3000

prefijo = "features_completos_"
sufijo = ".csv.gz"
clases = ["Arborio", "Basmati", "Ipsala", "Jasmine", "Karacadag"]
input_folder = "output/datasets/"
input_paths = [input_folder+prefijo+c+sufijo for c in clases]


sample_df = pd.DataFrame()
for clase in clases:
    path = input_folder+prefijo+clase+sufijo
    ids = np.random.choice(np.arange(0,DEFAULT_LENGTH,1), size=SAMPLE_LENGTH, replace=False)
    df = pd.read_csv(path) 
    df = df.loc[ids, :].reset_index(drop=True)
    sample_df = pd.concat([sample_df, df], axis=0)
    print(clase, sample_df.shape)

Arborio (3000, 4104)
Basmati (6000, 4104)
Ipsala (9000, 4104)
Jasmine (12000, 4104)
Karacadag (15000, 4104)


In [6]:
default_cols = ['image_id','class_name']
morphological_features = ['area','eccentricity','perimeter', 'orientation','axis_major_length','axis_minor_length']
conv2d_features = [str(i) for i in range(4096)]

sample_features = sample_df.drop(columns = default_cols + morphological_features)
sample_labels = sample_df.class_name.to_list()

In [7]:
# Uso PCA

ncols = sample_features.shape[1]
n_components = np.min([100, ncols])

scaler = StandardScaler()
scaled = scaler.fit_transform(sample_features)

pca = PCA(n_components=n_components)
scaled_pca = pca.fit(scaled)
Xpca = pca.transform(scaled)

exp_var = scaled_pca.explained_variance_ratio_

In [ ]:
KM_meta=KMedoids(n_clusters=4,metric='precomputed',init='k-medoids++').fit(d_meta)